# تصنيف البلاغات الأمنية باستخدام تعلم الآلة

## الهدف
بناء نموذج تعلم آلة يصنف البلاغات الأمنية حسب الأولوية:
- **عادي:** لا يحتاج استجابة فورية
- **عاجل:** يحتاج استجابة سريعة
- **طارئ:** يحتاج استجابة فورية

## المكتبات المستخدمة
- `pandas`: لمعالجة البيانات
- `scikit-learn`: لخوارزميات تعلم الآلة
- `matplotlib` و `seaborn`: للتصور البياني

In [1]:
import pandas as pd
import numpy as np
import json
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

print('✅ تم استيراد جميع المكتبات بنجاح')

✅ تم استيراد جميع المكتبات بنجاح


## 1. قراءة البيانات من ملف JSON

In [6]:
# قراءة ملف JSON
file_path = 'cleaned_reports_v2.csv'
data = pd.read_csv(file_path)
print(f'📊 عدد البلاغات: {len(data)}')
print(f'📋 الأعمدة: {list(data.columns)}')

📊 عدد البلاغات: 1000
📋 الأعمدة: ['id', 'report_category', 'report_type', 'report_status', 'location', 'report_date', 'report_time', 'report_description', 'injuries', 'weapon_involved', 'num_suspects', 'financial_loss', 'report_type_original', 'report_category_inherent', 'report_category_original']


## 2 عرض البيانات الأولى

In [7]:
data.head(5)
print(f'\n📈 معلومات البيانات:')
print(data.info())
data.describe()
data.isnull().sum()
data.isna().sum()
data


📈 معلومات البيانات:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 15 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   id                        1000 non-null   int64 
 1   report_category           1000 non-null   object
 2   report_type               1000 non-null   object
 3   report_status             1000 non-null   object
 4   location                  1000 non-null   object
 5   report_date               1000 non-null   object
 6   report_time               1000 non-null   object
 7   report_description        1000 non-null   object
 8   injuries                  1000 non-null   bool  
 9   weapon_involved           1000 non-null   bool  
 10  num_suspects              1000 non-null   int64 
 11  financial_loss            1000 non-null   int64 
 12  report_type_original      1000 non-null   object
 13  report_category_inherent  1000 non-null   object
 14  repo

,id,report_category,report_type,report_status,location,report_date,report_time,report_description,injuries,weapon_involved,num_suspects,financial_loss,report_type_original,report_category_inherent,report_category_original
0,1,طارئ,سرقة مسلحة,قيد التحقيق,الجوف,2023-07-31,2023-08-17 17:24:25,سرقة محل ذهب بسلاح,False,True,3,95207,سرقة,طارئ,عاجل
1,2,عادي,سرقة,مغلق,جيزان,2023-11-29,2023-08-17 16:08:41,سرقة من منزل,True,False,3,92735,مفقود,عادي,عاجل
2,3,عادي,حريق,مغلق,الدمام,2023-02-14,2023-08-17 18:27:27,حريق في محل تجاري,True,True,5,40622,إرهاب,طارئ,عادي
3,4,طارئ,غرق,مفتوح,عسير,2023-12-19,2023-08-17 09:59:10,غرق في البحر,True,True,4,60151,تخريب,طارئ,عاجل
4,5,عادي,تزوير,مغلق,جدة,2023-08-20,2023-08-17 10:37:15,تزوير وثائق,True,False,2,67400,مفقود,عادي,عاجل
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,996,عادي,سرقة,تم الحل,تبوك,2023-01-10,2023-08-17 11:02:49,سرقة محفظة من امرأة في الشارع الرئيسي,True,False,2,56087,حادث سير,عادي,عاجل
996,997,عادي,تهديد,قيد التحقيق,جدة,2023-11-04,2023-08-17 16:36:23,تهديد بالقتل عبر الهاتف,False,False,4,87513,حريق,عادي,عاجل
997,998,عادي,مفقود,تم الحل,نجران,2023-07-07,2023-08-17 00:27:36,فقدان طفل في السوق,False,False,3,40938,قتل,عاجل,طارئ
998,999,عادي,اعتداء,تم الحل,الدمام,2023-05-05,2023-08-17 00:20:59,اعتداء على موظف حكومي,True,False,3,50567,تخريب,عاجل,عادي


## 3. تحليل التوزيع

In [8]:
# توزيع التصنيفات
print('📊 توزيع الفئات:')
print(data['report_category'].value_counts())
print(f'\n📊 توزيع أنواع البلاغات:')
print(data['report_type'].value_counts())

📊 توزيع الفئات:
report_category
عادي    668
طارئ    197
عاجل    135
Name: count, dtype: int64

📊 توزيع أنواع البلاغات:
report_type
حريق            92
سرقة            84
اختطاف          61
إطلاق نار       60
حادث سير        58
عنف منزلي       57
مفقود           57
سرقة مسلحة      55
تهديد           54
تهريب مخدرات    54
اعتداء          50
تزوير           49
شجار            47
سرقة سيارة      46
غرق             45
تخريب           44
احتيال          44
اعتداء جنسي     43
Name: count, dtype: int64


## 4. معالجة البيانات (Feature Engineering)

In [12]:
# نسخ البيانات
df = data.copy()

# استخراج الساعة من report_time
df['hour'] = pd.to_datetime(df['report_time']).dt.hour
df['is_night'] = ((df['hour'] >= 22) | (df['hour'] <= 6)).astype(int)

# تحويل القيم المنطقية إلى أرقام
df['injuries'] = df['injuries'].astype(int)
df['weapon_involved'] = df['weapon_involved'].astype(int)

# ترميز النصوص
le_type = LabelEncoder()
le_status = LabelEncoder()
le_location = LabelEncoder()

df['report_type_encoded'] = le_type.fit_transform(df['report_type'])
df['report_status_encoded'] = le_status.fit_transform(df['report_status'])
df['location_encoded'] = le_location.fit_transform(df['location'])

# اختيار السمات (Features)
features = ['report_type_encoded', 'report_status_encoded', 'location_encoded', 
            'injuries', 'weapon_involved', 'num_suspects', 'financial_loss', 'is_night']

X = df[features]
y = df['report_category']

print('✅ تم معالجة البيانات بنجاح')
print(f'📊 عدد السمات: {len(features)}')
print(f'📊 عدد العينات: {len(X)}')

✅ تم معالجة البيانات بنجاح
📊 عدد السمات: 8
📊 عدد العينات: 1000


## 5. تقسيم البيانات

In [13]:
# تقسيم البيانات: 80% للتدريب، 20% للاختبار
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
print(f'📊 عدد عينات التدريب: {len(X_train)}')
print(f'📊 عدد عينات الاختبار: {len(X_test)}')
print(f'\n📊 توزيع الفئات في بيانات التدريب:')
print(y_train.value_counts())

📊 عدد عينات التدريب: 800
📊 عدد عينات الاختبار: 200

📊 توزيع الفئات في بيانات التدريب:
report_category
عادي    534
طارئ    158
عاجل    108
Name: count, dtype: int64


## 6. تدريب نموذج الغابة العشوائية (Random Forest)

In [14]:
# تدريب نموذج الغابة العشوائية
rf_model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
rf_model.fit(X_train, y_train)

# التنبؤ
y_pred_rf = rf_model.predict(X_test)

# حساب الدقة
rf_accuracy = accuracy_score(y_test, y_pred_rf)
rf_f1 = f1_score(y_test, y_pred_rf, average='weighted')

print('🌲 نموذج الغابة العشوائية (Random Forest)')
print(f'✅ الدقة: {rf_accuracy:.4f}')
print(f'✅ F1-Score: {rf_f1:.4f}')
print(f'\n📊 تقرير التصنيف:')
print(classification_report(y_test, y_pred_rf, digits=4))

🌲 نموذج الغابة العشوائية (Random Forest)
✅ الدقة: 0.8650
✅ F1-Score: 0.8500

📊 تقرير التصنيف:
              precision    recall  f1-score   support

        طارئ     0.8286    0.7436    0.7838        39
        عاجل     0.9167    0.4074    0.5641        27
        عادي     0.8693    0.9925    0.9268       134

    accuracy                         0.8650       200
   macro avg     0.8715    0.7145    0.7582       200
weighted avg     0.8677    0.8650    0.8500       200



## 7. تدريب نموذج KNN (K-Nearest Neighbors)

In [15]:
# تدريب نموذج KNN
knn_model = KNeighborsClassifier(n_neighbors=5)
knn_model.fit(X_train, y_train)

# التنبؤ
y_pred_knn = knn_model.predict(X_test)

# حساب الدقة
knn_accuracy = accuracy_score(y_test, y_pred_knn)
knn_f1 = f1_score(y_test, y_pred_knn, average='weighted')

print('🔍 نموذج KNN (K-Nearest Neighbors)')
print(f'✅ الدقة: {knn_accuracy:.4f}')
print(f'✅ F1-Score: {knn_f1:.4f}')
print(f'\n📊 تقرير التصنيف:')
print(classification_report(y_test, y_pred_knn, digits=4))

🔍 نموذج KNN (K-Nearest Neighbors)
✅ الدقة: 0.5600
✅ F1-Score: 0.5154

📊 تقرير التصنيف:
              precision    recall  f1-score   support

        طارئ     0.0435    0.0256    0.0323        39
        عاجل     0.2857    0.1481    0.1951        27
        عادي     0.6564    0.7985    0.7205       134

    accuracy                         0.5600       200
   macro avg     0.3285    0.3241    0.3160       200
weighted avg     0.4869    0.5600    0.5154       200



## 8. مقارنة النماذج

In [13]:
# مقارنة النماذج
comparison = pd.DataFrame({
    'النموذج': ['الغابة العشوائية', 'KNN'],
    'الدقة': [rf_accuracy, knn_accuracy],
    'F1-Score': [rf_f1, knn_f1]
})

print('📊 مقارنة النماذج:')
print(comparison.to_string(index=False))

# تحديد أفضل نموذج
best_model = 'الغابة العشوائية' if rf_accuracy > knn_accuracy else 'KNN'
print(f'\n🏆 أفضل نموذج: {best_model}')

📊 مقارنة النماذج:
         النموذج  الدقة  F1-Score
الغابة العشوائية  0.325  0.324903
             KNN  0.315  0.292474

🏆 أفضل نموذج: الغابة العشوائية


## 9. مصفوفة الالتباس (Confusion Matrix)

In [14]:
# حساب مصفوفة الالتباس
cm_rf = confusion_matrix(y_test, y_pred_rf)
cm_knn = confusion_matrix(y_test, y_pred_knn)

# رسم مصفوفة الالتباس
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# الغابة العشوائية
sns.heatmap(cm_rf, annot=True, fmt='d', cmap='Blues', ax=axes[0], 
            xticklabels=['عادي', 'عاجل', 'طارئ'],
            yticklabels=['عادي', 'عاجل', 'طارئ'])
axes[0].set_title('مصفوفة الالتباس - الغابة العشوائية', fontsize=12, fontweight='bold')
axes[0].set_ylabel('الفئة الفعلية')
axes[0].set_xlabel('الفئة المتنبأ بها')

# KNN
sns.heatmap(cm_knn, annot=True, fmt='d', cmap='Greens', ax=axes[1],
            xticklabels=['عادي', 'عاجل', 'طارئ'],
            yticklabels=['عادي', 'عاجل', 'طارئ'])
axes[1].set_title('مصفوفة الالتباس - KNN', fontsize=12, fontweight='bold')
axes[1].set_ylabel('الفئة الفعلية')
axes[1].set_xlabel('الفئة المتنبأ بها')

plt.tight_layout()
plt.show()

print('✅ تم رسم مصفوفات الالتباس')

✅ تم رسم مصفوفات الالتباس


## 10. أهمية السمات (Feature Importance)

In [ ]:
# الحصول على أهمية السمات من الغابة العشوائية
feature_importance = pd.DataFrame({
    'السمة': features,
    'الأهمية': rf_model.feature_importances_
}).sort_values('الأهمية', ascending=False)

print('📊 أهمية السمات:')
print(feature_importance.to_string(index=False))

# رسم أهمية السمات
plt.figure(figsize=(10, 6))
plt.barh(feature_importance['السمة'], feature_importance['الأهمية'], color='steelblue')
plt.xlabel('درجة الأهمية')
plt.title('أهمية السمات في نموذج الغابة العشوائية', fontsize=12, fontweight='bold')
plt.tight_layout()
plt.show()

## 11. اختبار التنبؤ على بلاغ جديد

In [15]:
# مثال على بلاغ جديد
new_report = pd.DataFrame({
    'report_type_encoded': [le_type.transform(['سرقة'])[0]],
    'report_status_encoded': [le_status.transform(['مفتوح'])[0]],
    'location_encoded': [le_location.transform(['الرياض'])[0]],
    'injuries': [1],  # نعم
    'weapon_involved': [1],  # نعم
    'num_suspects': [2],
    'financial_loss': [50000],
    'is_night': [1],  # ليل
    'hour': [23]
})

# التنبؤ باستخدام الغابة العشوائية
prediction_rf = rf_model.predict(new_report)[0]
probability_rf = rf_model.predict_proba(new_report)[0]

# التنبؤ باستخدام KNN
prediction_knn = knn_model.predict(new_report)[0]
probability_knn = knn_model.predict_proba(new_report)[0]

print('🔮 التنبؤ على بلاغ جديد:')
print(f'البلاغ: سرقة مسلحة في الرياض، مع إصابات، في الليل')
print(f'\n🌲 نموذج الغابة العشوائية:')
print(f'   التصنيف: {prediction_rf}')
print(f'   الاحتماليات: عادي={probability_rf[0]:.2%}, عاجل={probability_rf[1]:.2%}, طارئ={probability_rf[2]:.2%}')
print(f'\n🔍 نموذج KNN:')
print(f'   التصنيف: {prediction_knn}')
print(f'   الاحتماليات: عادي={probability_knn[0]:.2%}, عاجل={probability_knn[1]:.2%}, طارئ={probability_knn[2]:.2%}')

🔮 التنبؤ على بلاغ جديد:
البلاغ: سرقة مسلحة في الرياض، مع إصابات، في الليل

🌲 نموذج الغابة العشوائية:
   التصنيف: عادي
   الاحتماليات: عادي=27.00%, عاجل=35.00%, طارئ=38.00%

🔍 نموذج KNN:
   التصنيف: طارئ
   الاحتماليات: عادي=80.00%, عاجل=20.00%, طارئ=0.00%


## 12. الملخص والنتائج

In [16]:
print('='*60)
print('📊 ملخص النتائج')
print('='*60)
print(f'\n📈 عدد البلاغات المستخدمة: {len(data)}')
print(f'📈 عدد السمات: {len(features)}')
print(f'📈 نسبة التدريب/الاختبار: 80/20')
print(f'\n🌲 نموذج الغابة العشوائية:')
print(f'   - الدقة: {rf_accuracy:.4f} ({rf_accuracy*100:.2f}%)')
print(f'   - F1-Score: {rf_f1:.4f}')
print(f'\n🔍 نموذج KNN:')
print(f'   - الدقة: {knn_accuracy:.4f} ({knn_accuracy*100:.2f}%)')
print(f'   - F1-Score: {knn_f1:.4f}')
print(f'\n🏆 أفضل نموذج: {best_model}')
print(f'\n✅ تم بناء النماذج بنجاح!')
print('='*60)

📊 ملخص النتائج

📈 عدد البلاغات المستخدمة: 1000
📈 عدد السمات: 9
📈 نسبة التدريب/الاختبار: 80/20

🌲 نموذج الغابة العشوائية:
   - الدقة: 0.3250 (32.50%)
   - F1-Score: 0.3249

🔍 نموذج KNN:
   - الدقة: 0.3150 (31.50%)
   - F1-Score: 0.2925

🏆 أفضل نموذج: الغابة العشوائية

✅ تم بناء النماذج بنجاح!
